In [2]:
import sympy as sp
import re

def truss_stiffness_matrix(nodes, elements, EA, supports):
    """
    Assemble the global stiffness matrix for a 2D truss using sympy.

    Parameters:
        nodes (dict): Dictionary of node coordinates {node_id: (x, y)}
        elements (list): List of tuples (node1, node2) defining element connectivity
        A (dict): Dictionary of cross-sectional areas {element_id: A_value}
        E (dict): Dictionary of Young's modulus values {element_id: E_value}

    Returns:
        sympy.Matrix: Global stiffness matrix
    """
    num_nodes = len(nodes)
    dof = 2 * num_nodes  # Degrees of freedom (2 per node)
    K_global = sp.zeros(dof, dof)

    for i, (n1, n2) in enumerate(elements):
        x1, y1 = nodes[n1]
        x2, y2 = nodes[n2]
        L = sp.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)  # Element length
        c = (x2 - x1) / L  # Cosine of angle
        s = (y2 - y1) / L  # Sine of angle

        # Stiffness matrix for an element in local coordinates
        k_local = (EA[i] / L) * sp.Matrix([
            [c ** 2, c * s, -c ** 2, -c * s],
            [c * s, s ** 2, -c * s, -s ** 2],
            [-c ** 2, -c * s, c ** 2, c * s],
            [-c * s, -s ** 2, c * s, s ** 2]
        ])

        # Global DOF indices
        dof_indices = [2 * n1, 2 * n1 + 1, 2 * n2, 2 * n2 + 1]

        # Assemble into global stiffness matrix
        for r in range(4):
            for c in range(4):
                K_global[dof_indices[r], dof_indices[c]] += k_local[r, c]

        for i in supports:
            for j in range(K_global.shape[0]):
                K_global[i, j] = 0.
                K_global[j, i] = 0.
            K_global[i, i] = 1

    return K_global


# Nodes
def coordinates(n_panels, l, h):
    nodes = []
    for idx in range(n_panels + 1):
        x = idx * l
        nodes.append([x, 0.])
    for idx in range(n_panels + 1, 2 * n_panels):
        x = n_panels * l - l * (idx - n_panels)
        nodes.append([x, h])
    return nodes


def connectivity(n_panels):
    idx_bar = 0
    # Horizontal Elements
    elements = []
    for idx in range(n_panels):
        idx_start = idx
        idx_end = idx_start + 1
        elements.append([idx_start, idx_end])
        idx_bar += 1

    for idx in range(n_panels + 1, 2 * n_panels - 1):
        idx_start = idx
        idx_end = idx_start + 1
        elements.append([idx_start, idx_end])
        idx_bar += 1

    # Vertical Elements
    for idx in range(1, n_panels):
        idx_start = idx
        idx_end = 2 * n_panels - idx_start
        elements.append([idx_start, idx_end])
        idx_bar += 1

    # Diagonal Elements

    elements.append([0, 2 * n_panels - 1])
    idx_bar += 1

    for idx in range(1, n_panels // 2):
        idx_start = idx + 1
        idx_end = 2 * n_panels + 1 - idx_start
        elements.append([idx_start, idx_end])
        idx_bar += 1

    for idx in range(n_panels // 2, n_panels - 1):
        idx_start = idx
        idx_end = 2 * n_panels - 1 - idx_start
        elements.append([idx_start, idx_end])
        idx_bar += 1

    elements.append([n_panels, n_panels + 1])
    idx_bar += 1

    return elements

In [14]:
h = sp.Symbol('H', real=True, positive=True)
l = sp.Symbol('L', real=True, positive=True)

n_panels = 8
nodes = coordinates(n_panels, l, h)
elements = connectivity(n_panels)
EA = [sp.Symbol(f'EA{i:02d}', real=True, positive=True) for i in range(len(elements))]
K_global = truss_stiffness_matrix(nodes, elements, EA, (0, 1, 17))

#K_global = sp.simplify(K_global)

In [15]:
K_global

Matrix([
[1, 0,               0,       0,                                                0,                                       0,                                                0,                                       0,                                                                                 0,                                                                        0,                                                0,                                       0,                                                0,                                       0,               0,       0,                                       0, 0,                                                                        0,                                                                        0,                                                0,                                       0,                                                0,                                       0,               0,       0,            

In [25]:
print("def pratt_stiffness_matrix(H, L, EA):", end='\n\t')
print("H2 = H ** 2", end='\n\t')
print("L2 = L ** 2", end='\n\t')
print("D2_3_2 = (H ** 2 + L ** 2)**(3/2)", end='\n\t')
print("HL = H * L", end='\n\t')

print("k = torch.eye(32).repeat((len(EA), 1, 1))", end='\n\t')
print('', end='\n\t')
for i in range(K_global.shape[0]):
    for j in range(K_global.shape[1]):
        if not (K_global[i, j] == 0 or K_global[i, j] == 1):
            s = str(K_global[i, j])
            ea_set = set(re.findall(r'EA\d*', s))
            for ea in ea_set:
                idx = int(ea[2:])
                new_ea = f'EA[:,{idx}]'
                s = s.replace(ea, new_ea)

            s = s.replace("(H**2 + L**2)", "D2")
            s = s.replace("D2**(3/2)", "D2_3_2")
            s = s.replace("L**2", "L2")
            s = s.replace("H**2", "H2")
            s = s.replace("H*L", "HL")
            s = s.replace("L*H", "HL")

            print(f'k[:, {i}, {j}] = {s}', end='\n\t')
print('', end='\n\t')
print('return k')

def pratt_stiffness_matrix(H, L, EA):
	H2 = H ** 2
	L2 = L ** 2
	D2_3_2 = (H ** 2 + L ** 2)**(3/2)
	HL = H * L
	k = torch.eye(32).repeat((len(EA), 1, 1))
	
	k[:, 2, 2] = EA[:,0]/L + EA[:,1]/L
	k[:, 2, 4] = -EA[:,1]/L
	k[:, 3, 3] = EA[:,14]/H
	k[:, 3, 31] = -EA[:,14]/H
	k[:, 4, 2] = -EA[:,1]/L
	k[:, 4, 4] = EA[:,1]/L + EA[:,2]/L + EA[:,22]*L2/D2_3_2
	k[:, 4, 5] = -EA[:,22]*HL/D2_3_2
	k[:, 4, 6] = -EA[:,2]/L
	k[:, 4, 30] = -EA[:,22]*L2/D2_3_2
	k[:, 4, 31] = EA[:,22]*HL/D2_3_2
	k[:, 5, 4] = -EA[:,22]*HL/D2_3_2
	k[:, 5, 5] = EA[:,15]/H + EA[:,22]*H2/D2_3_2
	k[:, 5, 29] = -EA[:,15]/H
	k[:, 5, 30] = EA[:,22]*HL/D2_3_2
	k[:, 5, 31] = -EA[:,22]*H2/D2_3_2
	k[:, 6, 4] = -EA[:,2]/L
	k[:, 6, 6] = EA[:,2]/L + EA[:,3]/L + EA[:,23]*L2/D2_3_2
	k[:, 6, 7] = -EA[:,23]*HL/D2_3_2
	k[:, 6, 8] = -EA[:,3]/L
	k[:, 6, 28] = -EA[:,23]*L2/D2_3_2
	k[:, 6, 29] = EA[:,23]*HL/D2_3_2
	k[:, 7, 6] = -EA[:,23]*HL/D2_3_2
	k[:, 7, 7] = EA[:,16]/H + EA[:,23]*H2/D2_3_2
	k[:, 7, 27] = -EA[:,16]/H
	k[:, 7, 28] = EA[:,23]*HL/D

In [ ]:
code = sp.python(K_global)

for i in range(len(elements)):
    old_str = f"EA{i:02d}"
    new_str = f"EA[{i}]"
    code = code.replace(old_str, new_str)

rs = set(re.findall(r'Rational\(\d*,\s\d*\)', code))
for r in rs:
    a, b = r[len('Rational('):-1].split(', ')
    code = code.replace(r, f'({a}/{b})')

code = code.replace("(H**2 + L**2)", "D2")
code = code.replace("L**2", "L2")
code = code.replace("H**2", "H2")
code = code.replace("H*L", "HL")
code = code.replace("L*H", "HL")

n = len(elements) + 2
code = code.split("\n", n)[n]

code = code.replace("e = MutableDenseMatrix", 'k = np.array')
code = code.replace("], ", "],\n")

code = "\t" + code + "\n\treturn k"
code = "\tHL = H * L" + code
code = "\tD2 = H**2 + L**2" + code
code = "\tL2 = L**2" + code
code = "\tH2 = H**2" + code
code = "def pratt_stiffness_matrix(H, L, EA):\n" + code

print(code)

In [16]:
for i in range(K_global.shape[0]):
    for j in range(K_global.shape[1]):
        if not (K_global[i, j] == 0 or K_global[i, j] == 1):
            s = str(K_global[i, j])
            ea_set = set(re.findall(r'EA\d*', s))
            for ea in ea_set:
                idx = int(ea[2:])
                new_ea = f'EA[:,{idx}]'
                s = s.replace(ea, new_ea)

            s = s.replace("(H**2 + L**2)", "D2")
            s = s.replace("L**2", "L2")
            s = s.replace("H**2", "H2")
            s = s.replace("H*L", "HL")
            s = s.replace("L*H", "HL")

            print(f'\tk[:, {i}, {j}] = {s}')


	k[:, 2, 2] = EA[:,0]/L + EA[:,1]/L
	k[:, 2, 4] = -EA[:,1]/L
	k[:, 3, 3] = EA[:,14]/H
	k[:, 3, 31] = -EA[:,14]/H
	k[:, 4, 2] = -EA[:,1]/L
	k[:, 4, 4] = EA[:,1]/L + EA[:,2]/L + EA[:,22]*L2/D2**(3/2)
	k[:, 4, 5] = -EA[:,22]*HL/D2**(3/2)
	k[:, 4, 6] = -EA[:,2]/L
	k[:, 4, 30] = -EA[:,22]*L2/D2**(3/2)
	k[:, 4, 31] = EA[:,22]*HL/D2**(3/2)
	k[:, 5, 4] = -EA[:,22]*HL/D2**(3/2)
	k[:, 5, 5] = EA[:,15]/H + EA[:,22]*H2/D2**(3/2)
	k[:, 5, 29] = -EA[:,15]/H
	k[:, 5, 30] = EA[:,22]*HL/D2**(3/2)
	k[:, 5, 31] = -EA[:,22]*H2/D2**(3/2)
	k[:, 6, 4] = -EA[:,2]/L
	k[:, 6, 6] = EA[:,2]/L + EA[:,3]/L + EA[:,23]*L2/D2**(3/2)
	k[:, 6, 7] = -EA[:,23]*HL/D2**(3/2)
	k[:, 6, 8] = -EA[:,3]/L
	k[:, 6, 28] = -EA[:,23]*L2/D2**(3/2)
	k[:, 6, 29] = EA[:,23]*HL/D2**(3/2)
	k[:, 7, 6] = -EA[:,23]*HL/D2**(3/2)
	k[:, 7, 7] = EA[:,16]/H + EA[:,23]*H2/D2**(3/2)
	k[:, 7, 27] = -EA[:,16]/H
	k[:, 7, 28] = EA[:,23]*HL/D2**(3/2)
	k[:, 7, 29] = -EA[:,23]*H2/D2**(3/2)
	k[:, 8, 6] = -EA[:,3]/L
	k[:, 8, 8] = EA[:,3]/L + EA[:,4]/L + EA[: